In [2]:
import os

os.chdir("..")


In [45]:
from utils import load_dataset
from run_exp import get_instance
import optuna
from pytorch_lightning.utilities.model_summary import ModelSummary
import pandas as pd

def build_m3_and_m4_dictionary(dataset, group, freq):
    """
    Build a dictionary for M3 and M4 datasets.
    """
    dataset_dict = {
        "name": dataset,
        "directory": f"C:\\Users\\ricar\\mixture_of_experts_time_series\\data\\{dataset}\\",
        "group": group,
        "freq": freq,
    }
    return dataset_dict

def calculate_avg_active_params(nr_of_parameters, nr_of_experts, active_experts, model_name):
    """
    Calculate the average number of active parameters.
    """
    if not active_experts and not nr_of_experts:
        return nr_of_parameters
    
    if model_name == "nbeatsmoeshared":
        return round((nr_of_parameters * (active_experts + 1)) / (nr_of_experts + 1), 2)

    return round((nr_of_parameters  * active_experts) / nr_of_experts, 2)

list_models = ["nbeats", "nbeatsmoe", "nbeatsmoeshared", "nbeatsstackmoe"]


map_horizon_freq = {
    "M": 18,
    "Q": 8,
    "Y": 6,
}
list_datasets = [
    {"name": "gluonts_m1_quarterly"},
    {"name": "gluonts_m1_monthly"},
    {"name": "gluonts_m1_yearly"},
    {"name": "gluonts_tourism_monthly"},
    {"name": "gluonts_tourism_quarterly"},
    {"name": "gluonts_tourism_yearly"},
    build_m3_and_m4_dictionary("m3", "Monthly", "M"),
    build_m3_and_m4_dictionary("m3", "Quarterly", "Q"),
    build_m3_and_m4_dictionary("m3", "Yearly", "Y"),
    # build_m3_and_m4_dictionary("m4", "Monthly", "M"),
    # build_m3_and_m4_dictionary("m4", "Quarterly", "Q"),
    # build_m3_and_m4_dictionary("m4", "Yearly", "Y"),
]


In [11]:
STORAGE = "sqlite:///c:/Users/ricar/mixture_of_experts_time_series/db/study_nbeats_blcs.db"

In [46]:
summaries_pd = pd.DataFrame(
    {
        "model_name": [],
        "dataset": [],
        "freq": [],
        "nr_of_parameters": [],
        "nr_experts": [],
        "top_k": [],
        "average_active_parameters": [],
    }
)

for dataset in list_datasets:
    Y_ALL = load_dataset(
        dataset["name"],
        dataset
    )
    horizon = None
    for model_name in list_models:
        if type(Y_ALL) == tuple:
            Y_ALL, horizon, n_lags, dataset["group"], _ = Y_ALL
            dataset["freq"] = dataset["group"]
            dataset["name"] = dataset["name"].replace(" ", "_")

        if horizon == None:
            horizon = map_horizon_freq[dataset["freq"]]

        study_name = f"{model_name}_{dataset['name']}_{dataset['group']}_{horizon}"
        print(study_name)

        study = optuna.load_study(
                        study_name=study_name,
                        storage=STORAGE,
        )

        # initialize the model with the best trial parameters and print the number of parameters
        best_params = study.best_params

        # some models where run with a different number of units in the mlp
        # manely to ensure that we could run the exp in a reasonable time
        if model_name == "nbeatsmoe" or model_name == "nbeatsmoeshared" and \
           (dataset["name"] == "gluonts_tourism_yearly" \
            or dataset["name"] == "gluonts_m1_quarterly" \
            or dataset["name"] == "gluonts_m1_monthly" \
            or dataset["name"] == "gluonts_tourism_monthly"):
            if best_params["mlp_units"][0][0] == 512 or best_params["mlp_units"][0][0] == 256:
                best_params["mlp_units"] = [[128, 128], [128, 128], [128, 128]]

        model = get_instance(model_name, best_params, horizon=horizon)

        nr_of_parameters = sum(p.numel() for p in model.parameters())
        nr_experts = best_params.get("nr_experts", None)
        top_k = best_params.get("top_k", None)

        summaries_pd = pd.concat(
            [
            summaries_pd,
            pd.DataFrame(
                {
                "model_name": [model_name],
                "dataset": [dataset["name"]],
                "freq": [dataset["freq"]],
                "nr_of_parameters": [nr_of_parameters],
                "nr_experts": [nr_experts],
                "top_k": [top_k],
                "average_active_parameters": [
                    calculate_avg_active_params(
                        nr_of_parameters, nr_experts, top_k, model_name
                    )    
                ],
                }
            ),
            ],
            ignore_index=True,
        )

        # print the model summary
        summary = ModelSummary(model)
        print(summary)

        del model, study, best_params, summary

# store the dataframe in a csv file
summaries_pd.to_csv(
    "c:/Users/ricar/mixture_of_experts_time_series/results/model_summaries.csv",
    index=False,
)
        


INFO:root:using dataset already processed in path C:\Users\ricar\.gluonts\datasets\m1_quarterly.


Loading m1_quarterly dataset...


INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 3


nbeats_gluonts_m1_quarterly_Q_2
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 152 K  | train
-------------------------------------------------------
152 K     Trainable params
30        Non-trainable params
152 K     Total params
0.610     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode
nbeatsmoe_gluonts_m1_quarterly_Q_2
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 1.2 M  | train
-------------------------------------------------------
1.2 M     Traina

INFO:lightning_fabric.utilities.seed:Seed set to 20
INFO:lightning_fabric.utilities.seed:Seed set to 18
INFO:root:using dataset already processed in path C:\Users\ricar\.gluonts\datasets\m1_monthly.


...

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 12.3 K | train
4 | gate         | Sequential    | 259    | train
-------------------------------------------------------
12.3 K    Trainable params
272       Non-trainable params
12.5 K    Total params
0.050     Total estimated model params size (MB)
34        Modules in train mode
0         Modules in eval mode
Loading m3_monthly dataset...


INFO:lightning_fabric.utilities.seed:Seed set to 7
INFO:lightning_fabric.utilities.seed:Seed set to 9


nbeats_m3_Yearly_6
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 1.3 K  | train
-------------------------------------------------------
1.2 K     Trainable params
156       Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode
nbeatsmoe_m3_Yearly_6
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 4.0 K  | train
-------------------------------------------------------
3.8 K     Trainable params
156       Non-t

INFO:lightning_fabric.utilities.seed:Seed set to 16
INFO:lightning_fabric.utilities.seed:Seed set to 10


  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 10.8 K | train
-------------------------------------------------------
10.7 K    Trainable params
156       Non-trainable params
10.8 K    Total params
0.043     Total estimated model params size (MB)
211       Modules in train mode
0         Modules in eval mode
nbeatsstackmoe_m3_Yearly_6
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 3.6 K  | train
4 | gate         | Sequential    | 138    | train
-------------------------------------------------------
3.6 K 